In [1]:
import os
import csv
import json
from google_images_download import google_images_download
from bs4 import BeautifulSoup
import shutil

In [3]:
def cooking_time(soup):
    return soup.find('dd',{'class':'facts__value svelte-1dqq0pw'}).text_strip

In [4]:
def getcalories(soup):
    return soup.find('p',{'class':'recipe-nutrition__item bold svelte-epeb0m'}).text_strip